In [8]:
import json
import numpy as np
import networkx as nx
import tqdm
import os
import yaml

name = "start_topo16_100000000000"
dir_name = "star16"

topology = "STAR"


# read topo data
with open(f'data/{name}.json','r') as f:
    data = json.load(f)

# get nodes and egdes
nodes, edges = data['nodeList'], data['edgeList']
NUM_NODES, NUM_EDGES = len(nodes), len(edges)

# creating some usefull maps to get information regarding the nodes
try:
    new_to_old = {}
    for node in nodes:
        new_to_old[node['id']] = node['old_id']
except KeyError:
    print("FAT TREE HAS NO OLD_ID... ingoring")

id_to_node = {}
for node in nodes:
    id_to_node[node['id']] = node

port_map = {}
for node in nodes:
    port_map[node['id']] = {}
    for port in node['portList']:
        port_map[node['id']][port['id']] = port


FAT TREE HAS NO OLD_ID... ingoring


In [9]:
g = nx.DiGraph()

connection_info = {node['id']: {} for node in nodes}

for edge in edges:
    src, src_port, dest, dest_port = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']
    src_bw, dest_bw = port_map[src][src_port]['bw'], port_map[dest][dest_port]['bw']
    
    connection_info[src][dest] = port_map[src][src_port]

    g.add_edge(src, dest)

print(g)

DiGraph with 17 nodes and 32 edges


In [10]:
if topology == "ZTE":
    # get the nodes of the subgraphs in the topology
    subgraphs = list(nx.strongly_connected_components(g))

    # get the topology of selected subgraph
    g = g.subgraph(subgraphs[1])

In [11]:
import shutil

save_dir = os.getcwd() + f"/topologies/{dir_name}"

try:
    os.makedirs(save_dir, exist_ok=True)
except:
    shutil.rmtree(save_dir)
    os.mkdir(save_dir)

shortest_paths = nx.all_pairs_shortest_path(g) # using Floyd-Warshall

'''
    returns list of tuples: [(source, {dest1: path, dest2: path ...})]
'''
for paths in tqdm.tqdm(shortest_paths, desc=f"creating routing tables", unit="iteration"): # This should also be NUM_NODES but for now I am only computing the table for 1 node
    source = paths[0] # getting the source node
    # initialising the switch infro struct
    switch_info = { 'qos': None,
                    'terminals': None,
                    'type': id_to_node[source]['deviceLevel'],
                    'ports': {x['id']: x['bw'] for x in port_map[source].values()},
                    'routing': {}
                }
    # adding the routing information for all possible destinations
    for dest, path in paths[1].items():
        if source != dest:
            next_hop = path[1]
            switch_info['routing'][dest] = {'next_hop': next_hop, 'port': connection_info[source][next_hop]['id']}
            
    # writting the yaml switch file
    with open(f'topologies/{dir_name}/{source}.yaml','w') as f:
        yaml.dump(switch_info, f)



creating routing tables: 17iteration [00:00, 376.30iteration/s]


--------------------------------------------- BELOW ARE TEST FUCTIONS ETCCC -----------------------------------------


In [6]:
cnt = 0
for paths in tqdm.tqdm(shortest_paths, desc=f"creating routing tables", unit="iteration"): # This should also be NUM_NODES but for now I am only computing the table for 1 node
    source = paths[0]
    switch_info = {'qos': None, 'terminals': None, 'type': id_to_node[source]['deviceLevel'],'ports': {x['id']: x['bw'] for x in port_map[source].values()}, 'routing': {}}
    for dest, path in paths[1].items():
        if source != dest:
            next_hop = path[1]
            switch_info['routing'][dest] = {'next_hop': next_hop, 'port': connection_info[source][next_hop]['id']}

    cnt += 1

    if cnt == 100:
        break
    # with open(f'first_subgraph/{source}.yaml','w') as f:
    #     yaml.dump(switch_info, f)


creating routing tables: 0iteration [00:00, ?iteration/s]


In [7]:
for source in tqdm.tqdm(g, desc="Calculating shortest paths", unit="iteration"): # This should also be NUM_NODES but for now I am only computing the table for 1 node
    switch_info = {'qos': None, 'terminals': None, 'ports': {x['id']: x['bw'] for x in port_map[source].values()}, 'routing': {}}
    for dest in g:
        if source != dest:
            route = nx.shortest_path(g, source, dest)
            next_hop = route[1]
            switch_info['routing'][dest] = {'next_hop': next_hop, 'port': connection_info[source][next_hop]['id']}

    with open(f'second_subgraph/{source}.yaml','w') as f:
        yaml.dump(switch_info, f)


Calculating shortest paths:   0%|                                                   | 0/17 [00:00<?, ?iteration/s]


FileNotFoundError: [Errno 2] No such file or directory: 'second_subgraph/1.yaml'

In [ ]:
print(connection_info[0][942])

{'id': 'TwentyFiveGigE-0/1/1/28', 'bw': 25000000000.0}


In [ ]:
import yaml

for src, row in enumerate(routing_table):
    switch_info = {'qos': None, 'terminals': None, 'ports': {x['id']: x['bw'] for x in port_map[src].values()}, 'routing': {}}
    
    for dest, path in enumerate(row):
        if source != dest:
            next_hop = path[1]
            switch_info['routing'][dest] = {'next_hop': next_hop, 'port': connection_info[src][next_hop]['id']}

with open('first_subgraph/test.yaml','w') as f:
    yaml.dump(switch_info, f)

Below is a test to connect all the disconnetec graphs to check if the routing tables will be complete

In [ ]:
'''
     TEST: 
         Using a undirected graph - find min_span_trees for all subgraphs and connect
        them to create one connected graph

        NOTE: This can only be done for a undirected graph
'''
g = nx.from_numpy_array(adj, create_using=nx.Graph) # Must be undirected graph
print(g)
cc = list(nx.connected_components(g))
print(f"Number of subgraphs: {len(cc)}")

# compute MST
minimum_spanning_trees = [nx.minimum_spanning_tree(g.subgraph(comp)) for comp in cc]
# compute the connections required to connect all MSPS
edges_to_connect = []
for i in range(len(minimum_spanning_trees) - 1):
    u, v = min(minimum_spanning_trees[i].nodes), min(minimum_spanning_trees[i + 1].nodes)
    edges_to_connect.append((u, v))
    
# actually connect the min_span_trees 
for s, d in edges_to_connect:
    g.add_edge(s, d)

cc = list(nx.connected_components(g))
print(f"Number of subgraphs (AFTER CONNECTING MSP's): {len(cc)}")

Graph with 9183 nodes and 10740 edges
Number of subgraphs: 32
Number of subgraphs (AFTER CONNECTING MSP's): 1


In [ ]:
routing_table = []

for source in tqdm.tqdm(range(1), desc="Calculating shortest paths", unit="iteration"): # This should also be NUM_NODES but for now I am only computing the table for 1 node
    row = []
    for dest in range(NUM_NODES): # Routing Table is NUM_NODES x NUM_NODES
        route = None   
        if source != dest:
            try:
                route = nx.shortest_path(g, source, dest)
            except:
                pass
        row.append(route)
        
    routing_table.append(row)

Calculating shortest paths:   0%|          | 0/1 [00:00<?, ?iteration/s]Calculating shortest paths: 100%|██████████| 1/1 [00:01<00:00,  1.53s/iteration]


In [ ]:
num_none = -1 # ignoring path for self

for source, row in enumerate(routing_table):
    for dest, path in enumerate(row):
        num_none += 1 if path is None else 0
        print(f"{source} -> {dest}: {path}")

print(f'{num_none} out of {NUM_NODES-1} none existent paths')

0 -> 0: None
0 -> 1: [0, 1]
0 -> 2: [0, 942, 9104, 7856, 1070, 204, 2649, 5884, 2]
0 -> 3: [0, 942, 8136, 4629, 5711, 6083, 2145, 7599, 3]
0 -> 4: [0, 942, 8136, 4629, 3461, 3581, 3488, 7721, 4604, 4549, 6507, 5810, 2221, 4]
0 -> 5: [0, 1, 288, 1677, 4830, 7016, 1125, 8789, 7200, 4007, 1722, 4996, 3774, 5866, 5]
0 -> 6: [0, 942, 8136, 4629, 3461, 3581, 3488, 7721, 230, 8141, 2978, 5480, 7035, 6861, 6]
0 -> 7: [0, 942, 8136, 4629, 3461, 3581, 3488, 7721, 230, 2371, 1255, 6598, 7]
0 -> 8: [0, 942, 8136, 4629, 3461, 3581, 1899, 2172, 1983, 3705, 8]
0 -> 9: [0, 942, 8136, 4629, 3461, 3581, 813, 299, 2601, 2288, 4309, 3688, 5610, 4851, 9088, 4514, 5315, 1440, 2903, 9]
0 -> 10: [0, 1, 10]
0 -> 11: [0, 1, 288, 1677, 4830, 7016, 7975, 1197, 303, 7071, 11]
0 -> 12: [0, 1, 288, 1677, 4830, 7016, 7975, 143, 4807, 755, 2496, 5916, 12]
0 -> 13: [0, 942, 8136, 4629, 3461, 3581, 813, 299, 2601, 6301, 881, 4626, 7555, 372, 13]
0 -> 14: [0, 1, 288, 598, 14]
0 -> 15: [0, 942, 8136, 4629, 3461, 3581, 348

Below are the scripts to identify problematic nodes/edges to send to ZTE

In [ ]:
import json

# read topo data
with open('resultData_updated_again.json', 'r') as f:
    data = json.load(f)

# get nodes and egdes
nodes, edges = data['nodeList'], data['edgeList']

# create a hash map of node_id: ports to eashily access port information using the ids in the edges
nodes_hash = {}
for node in nodes:
    nodes_hash[node['id']] = node

# removing nodes that are never a source in the edges (to prevent generation of a discnonnected graph)
removed_nodes = set()  # so we can remove edges that refer to them as dest nodes later

cleaned_nodes = []
while nodes:
    # get node
    node = nodes.pop()
    # if node has no ports - remove node
    if not node['portList']:
        removed_nodes.add(node['id'])
    else:
        # check if node is part of the topology
        is_src = False
        for edge in edges:
            if edge['srcNode'] == node['id']:
                is_src = True
                break
        # if the node conncets to another as a src node keep it
        if is_src:
            cleaned_nodes.append(node)

for n in removed_nodes:
    appears_as_source = len([edge for edge in edges if edge['srcNode'] == n])
    print(f"ID: {n} PORTS: {nodes_hash[n]['portList']} appears as source: {appears_as_source} type: {nodes_hash[n]['deviceLevel']}")

# same_ports = []
# for edge in edges
#  if edge['srcPort'] == edge['destPort']:
#     same_ports.append(edge)

# print(len(same_ports))

ID: b13773e5-fda5-42d7-a712-d44bbeded0da PORTS: [] appears as source: 0 type: Access
ID: d148424d-31dd-4447-98bc-c7bc3f6d4a9a PORTS: [] appears as source: 0 type: Access
ID: 9d2d0640-58ba-4224-a5f8-215c806f295a PORTS: [] appears as source: 0 type: Access
ID: 81a8fdc0-e456-4082-9180-29f192e1277d PORTS: [] appears as source: 0 type: Access
ID: 34f71db3-954d-476b-84f9-f32474caac06 PORTS: [] appears as source: 0 type: Access
ID: 7f0b0ec1-953f-4b09-9270-24678af99a19 PORTS: [] appears as source: 0 type: Access
ID: eeaeedc1-1fd0-4b6d-ac1d-8aedc44e08d7 PORTS: [] appears as source: 0 type: Access
ID: fdbc704e-d69a-445f-9cde-e74810bc3a75 PORTS: [] appears as source: 0 type: Access
ID: 8cba0600-87e6-4731-9494-6b9311bb2611 PORTS: [] appears as source: 0 type: Access
ID: 90544834-8bd5-4996-9f3f-0d1c68b42514 PORTS: [] appears as source: 0 type: Access
ID: 4bd63e74-6daf-4b19-a2d5-525fce9a54cd PORTS: [] appears as source: 0 type: Access
ID: 4e86a0e8-273e-4357-90e8-442a703cb736 PORTS: [] appears as sou

In [ ]:
'''
    CHECK IF NEW DATA EDGES HAVE CORRECT PORTS

    RESULT: They do
'''
# read topo data
with open('resultData_updated_again.json', 'r') as f:
    data = json.load(f)

# get nodes and egdes
nodes, edges = data['nodeList'], data['edgeList']

# create a hash map of node_id: ports to easily get the port names of a node
node_to_port_map = {}
for node in nodes:
    list_of_ports = [port['id'] for port in node['portList']]
    node_to_port_map[node['id']] = list_of_ports

# for every edge check if src and dest nodes contain the ports scpecified for that edge
for edge in edges:
    src, src_port, dest, dest_port = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']

    if src_port not in node_to_port_map[src]:
        print("SRC MISSING:", src, port, node_to_port_map[src])

    if dest_port not in node_to_port_map[dest]:
        print("DEST MISSING:", dest, port, node_to_port_map[dest])


In [ ]:
'''
    CHECK IF ALL EDGES HAVE AN EVIL TWIN :P
'''
for edge in edges:
    has_twin = False
    for other_edge in edges:
        if other_edge['srcNode'] == edge['destNode'] and other_edge['destNode'] == edge['srcNode']:
            has_twin = True
            break
    if not has_twin:
        print('HAS NO TWIN:', edge)

In [ ]:
'''
    CHECK IF NEW DATA HAVE NODES THAT ARE NEVER A SOURCE
'''
import json 

# read topo data
with open('resultData_updated_again.json', 'r') as f:
    data = json.load(f)

# get nodes and egdes
nodes, edges = data['nodeList'], data['edgeList']

for node in nodes:
    for edge in edges:
        is_src = False
        src, src_port, dest, dest_port = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']
        if src == node['id']:
            is_src = True
            break

    is_dest = False
    for edge in edges:
        src, src_port, dest, dest_port = edge['srcNode'], edge['srcPort'], edge['destNode'], edge['destPort']
        if dest == node['id']:
            is_dest = True
            break
        
    if not all([is_dest, is_dest]):
        print(node['id'], f"src: {is_src} | dest: {is_dest}", "PORTS:", node['portList'])


eeaeedc1-1fd0-4b6d-ac1d-8aedc44e08d7 src: False | dest: False PORTS: []
20b71a6a-04bb-443d-9aaa-784b50de695b src: False | dest: False PORTS: []
d5938cbc-9b7e-43dc-b46f-a10c1dc436b0 src: False | dest: False PORTS: []
b77bae6d-b61e-4a34-85f4-76d049cb8889 src: False | dest: False PORTS: []
d129b375-026c-4a1a-9eb4-39b6744b37b6 src: False | dest: False PORTS: []
18bdd3cf-6f93-4c88-829f-8039d2c7499e src: False | dest: False PORTS: []
3d0aaee2-4643-4c6e-b5d5-5d97014d5051 src: False | dest: False PORTS: []
ae39ba35-76ea-406b-888d-de775612926e src: False | dest: False PORTS: [{'id': 'TwentyFiveGigE-0/1/1/32', 'bw': 25000000000.0}]
78344a18-8edb-45ff-9a08-f249c8014dd8 src: False | dest: False PORTS: []
cc055a56-abf2-4726-874f-460c68d79b93 src: False | dest: False PORTS: []
f0924b32-8bb2-4515-91bb-b1ef837a2206 src: False | dest: False PORTS: []
9d2d0640-58ba-4224-a5f8-215c806f295a src: False | dest: False PORTS: []
8cba0600-87e6-4731-9494-6b9311bb2611 src: False | dest: False PORTS: []
9461b8c4-c

In [ ]:
# create directed graph from the adj. matrix
g = nx.from_numpy_array(adj, create_using=nx.Graph)

cc = list(nx.connected_components(g))
print(f"Number of subgraphs: {len(cc)}")

for idx, comp in enumerate(cc):
    print(f'subgraph {idx}: {[new_to_old[x] for x in comp ]}\n\n')
    

Number of subgraphs: 32
subgraph 0: ['25c59b4b-b33b-4b46-98ee-3e8ba53417a7', '856bc788-e3c7-4329-b4e6-0a6e532adc7f', 'bbc0779d-3087-4a6d-964d-34ce06932019', '6d1df5bd-6473-4658-8b28-4eb1120e3e51', '604ed56b-21ea-4e11-a92c-ceb0d639e60c', '90d1941a-bc7d-437e-bf69-88d6461ef80a', 'ec6e2aa6-89f6-4f04-b855-e1d88843de06', 'edfdb54a-8f9e-495d-9df6-f5bebefa005d', 'e5ae3b7b-82c8-4ab5-8732-e3a2fc10ccad', '6c9be397-0d9f-43df-9e3f-ca7169cf3989', '623938d0-dcf4-4905-84a5-45eb8e6c7079', 'de6c9e3f-f9e8-469d-a9f7-3e914ae7e6c7', '1f12e0b2-3e81-42fe-8877-598d54818465', 'fd8c7b2c-a12a-4ca0-ae9e-aa37aab2e9a5', '74ee3652-7692-4154-9596-2d733e9b983e', '60a21d7e-7fe5-4239-9b20-19a294e4d5f6', '31390ace-a4ca-4d53-8693-66111d0d8f28', 'be61315f-9ca5-4877-b84b-a7d8fee9102c', '8586c9df-2ea6-4365-b61d-36f26b7c391a', '12499161-2782-428e-99f3-2ff7b388aa78', '58d768c5-bc81-40f3-a7b8-054a35eadf22', '4298d7fc-dc55-481a-8b28-a9419872fff1', 'f0ca39c1-e435-4292-9f3f-2bcceaa4d9f4', '378459f3-36e7-4968-b8b5-83b32e9c7325', '92